In [17]:
"""
Statistical estimates and inference of Rayleigh distribution in gun target data
"""
import numpy as np
import math
from scipy.stats.distributions import chi2

In [146]:
class group:
    """Estimators of Rayleigh sigma and confidence from shot groups"""
    cG = {}  # Memoize cG correction factor

    def __init__(self, n: int, shots: list[list[float]]=[], runOrderStatistics: bool=False):
        """
        :param n: Number of shots
        :param shots: List of (x,y) coordinates of center of each shot
            If no data provided then all simulation samples will be drawn from X,Y~N(0,1)
            in which case we know the true parameter we're estimating is sigma=1        
        """
        g = 1
        self.n = n
        self.degrees = 2*(n-g)
        if n not in group.cG:
            d = self.degrees + 1
            group.cG[n] = 1/math.exp(math.log(math.sqrt(2/(d-1)))+math.lgamma(d/2)-math.lgamma((d-1)/2))

        if len(shots) > 0:
            self.shots = shots
        else:  # If we didn't receive a group of shots then create one from standard normal
            self.shots = [np.random.normal(size=2) for i in range(0,n)]

        self.sampleCenter = np.mean(self.shots, axis=0)
        self.centeredShots = np.add(self.shots, -self.sampleCenter)  # Shots adjusted so their centroid = (0,0)
        self.radii = np.linalg.norm(self.centeredShots, axis=-1)  # List of radii from sample center
        self.sumR2 = np.dot(self.radii, self.radii)
        self.sigmaEstimate = group.cG[n] * math.sqrt(self.sumR2 / self.degrees)  # Unbiased Rayleigh parameter estimate
        self.sigmaUpperConfidence = math.sqrt(self.sumR2 / chi2.ppf(0.05, self.degrees))  # Upper bound of 90% confidence interval

        if runOrderStatistics: self.sigmaFromOrderStatistics()

    def sigmaFromOrderStatistics(self):
        # Ref http://ballistipedia.com/index.php?title=File:Order_Statistics_of_Exponential_Distribution_in_Censored_Samples.pdf
        self.sortedShots = np.argsort(self.radii)  # Index of shot by radius, small to large
        if self.n == 3:
            self.orderSampleR2 = 0.5455 * self.Rstat(3)**2
        elif self.n == 5:
            self.orderSampleR2 = 0.7792 * self.Rstat(4)**2
        elif self.n == 10:
            self.orderSampleR2 = 0.4913 * self.Rstat(6)**2 + 0.3030 * self.Rstat(9)**2
        else:
            raise Exception(f"No order statistics defined for n={self.n}")
        self.orderSampleSumR2 = self.n * self.orderSampleR2
        self.sigmaOrderStatEstimate = group.cG[self.n] * math.sqrt(self.orderSampleSumR2 / self.degrees)
        self.sigmaOrderStatUpperConfidence = math.sqrt(self.orderSampleSumR2 / chi2.ppf(0.05, self.degrees))  # Upper bound of 90% confidence interval -- OS estimate is 85% efficient
        return self.sigmaOrderStatEstimate

    def Rstat(self, m: int):
        """Return mth smallest radii -- i.e., order statistic R(m)"""
        if m > self.n:
            raise Exception(f"Tried to access order statistic {m} on group size {self.n}")
        return self.radii[self.sortedShots][m-1]

# Simulation

In [147]:
sim = [group(5, runOrderStatistics=True) for i in range(0,100000)]

In [148]:
sigmaEstimates = [s.sigmaEstimate for s in sim]
sigmaUppers = [s.sigmaUpperConfidence for s in sim]
sigmaOSestimates = [s.sigmaOrderStatEstimate for s in sim]
sigmaOSuppers = [s.sigmaOrderStatUpperConfidence for s in sim]

In [149]:
print(f"Sigma   mean={np.mean(sigmaEstimates):.3f}\tmedian={np.median(sigmaEstimates):.3f}\tVar={np.var(sigmaEstimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaUppers if x < 1)/len(sigmaUppers):.0%}")
print(f"SigmaOS mean={np.mean(sigmaOSestimates):.3f}\tmedian={np.median(sigmaOSestimates):.3f}\tVar={np.var(sigmaOSestimates):.3f}\t90%Confidence<Parameter={sum(1 for x in sigmaOSuppers if x < 1)/len(sigmaOSuppers):.0%}")
print(f"\tOrder Stat Efficiency = {(np.var(sigmaEstimates)/np.var(sigmaOSestimates)):.3f}")

Sigma   mean=1.001	median=0.989	Var=0.064	90%Confidence<Parameter=5%
SigmaOS mean=1.012	median=0.990	Var=0.082	90%Confidence<Parameter=6%
	Order Stat Efficiency = 0.786


# Tests

In [76]:
testData = [[ 0.40,  0.09],
[  0.46,  0.19],
[ -0.07, -0.30],
[  0.60,  0.32],
[ -0.07, -0.07],
[  0.69, -0.14],
[  0.22, -0.03],
[ -0.05, -0.37],
[  0.63, -0.23],
[  0.32,  0.32],
[ -0.06, -0.25],
[  0.59, -0.29],
[  0.36,  0.02],
[  0.11,  0.19],
[  0.34, -0.29],
[  0.20, -0.66],
[  0.13, -0.01],
[  0.47, -0.23],
[  0.14, -0.15],
[  0.16, -0.32],
]

In [150]:
g1 = group(10, testData[0:10])
g1.sigmaFromOrderStatistics()
print(g1.sampleCenter)
print(g1.sigmaEstimate)
print(g1.sigmaUpperConfidence)
print(g1.sigmaOrderStatEstimate)
print(g1.sigmaOrderStatUpperConfidence)

[ 0.313 -0.022]
0.27638744568795726
0.37738311822923054
0.2838577421986046
0.38758316108670493


In [152]:
t = group(20, shots=testData)
print(t.sigmaEstimate)
print(t.sigmaUpperConfidence)

0.2497434511422087
0.3065983223591547
